# Cats and Dogs in Cortex Redux

### Introduction

This is a tutorial based on the fast.ai Practical Deep Learning course and their [Cats and Dogs Redux](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/dogs_cats_redux.ipynb)

This Jupyter Notebook uses the [lein-jupyter plugin](https://github.com/didiercrunch/lein-jupyter) to be able to execute Clojure code in
project setting. The first time that you run it you will need to install the kernal with `lein jupyter install-kernel`.
After that you can open the notebook in the project directory with `lein jupyter notebook`

The Deep Learning library in Clojure that we are using is called [Cortex](https://github.com/thinktopic/cortex)

The goal of this is to take the dataset from the [Kaggle Cats and Dogs Redux Competiton](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)
and train a model. We will then run the model on the Kaggle testing data and submit the results to see how we compare.

### Setup

You will need to do a bit of setup first. Download the `train.zip` and `test.zip` from the [Kaggle Data page](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data).

Make a `/data` directory in the project directory and put the `train.zip` file there and unzip it.

You should now have a directory structure that looks like `data/train` with inside the directory lots of pictures like `cat.156.jpg` and `dog.1.jpg`

The next thing that we need to do is to run the `.get-resnet50.sh` in the root of the project. What this is going to do is download a pretrained model that is meant to classify 1000 images types from the [ImageNet competition](http://www.image-net.org/challenges/LSVRC/). This model and its weights have been translated into a Cortex network format `nippy` 

In [16]:
(require '[clojure.java.shell :as shell])

(-> (shell/sh "./get-resnet50.sh") :out println)

In [17]:
(-> (shell/sh "ls" "models") :out println)

resnet50.nippy



In [19]:
(-> (shell/sh "ls" "-d" "data/train") :out println)

data/train



### Plan of Action

1. Get all the cats and dogs pictures in the right directory format for training
2. Train the model with all but the last layer in the RESNET model. The last layer we are going to replace with our own layer that will _finetune_ it to classify only cats and dogs
3. Run the test data and come up with a spreadsheet of results to submit to Kaggle.


### Step 1 - Getting the data in the right format

To do this, let's require some of the functions in the core namespace of this project. In fact, let's go ahead and create a proper namespace for this worksheet and require in the libs we need.

In [20]:
(ns cats-dogs-cortex-redux.notebook
    (:require [cats-dogs-cortex-redux.core :as cats-dogs]
              [clojure.java.shell :as shell]))

Now we are trying to get the cats and dogs data that's currently in the `data/train` directory into the form of directories that will look like this:

```
-data
  -cats-dogs-training
      -cat
          1110.png
          ...
      -dog
          12416.png
          ...
  -cats-dogs-testing
      -cat
          11.png
          ...
      -dog
          12.png
          ...
```

One of the main things we want to do is take all the images in the `data/train` directory, shuffle them, and then split them into 85% of the pictures for training and 15% of the pictures for validation testing. We also want to have a directory structure so that only the cat pictures are in the cat directory and dog pictures in the dog directory. Also all of the pictures should be resized to 224x224 to match up with the RESNET model's input, (it's expecting a 224x224 image).

There is a `build-image-data` function in the cats-dogs namespace that will do this for us.


In [21]:
(cats-dogs/build-image-data)

Building the image data with a test-training split of  0.85
training files  21250 testing files  3750


In [24]:
(-> (shell/sh "ls" "data/cats-dogs-training/") :out println)

cat
dog



In [26]:
(-> (shell/sh "ls" "data/cats-dogs-testing/") :out println)

cat
dog



### Step 2  - Training the Model

The main function our code calls to create the model is this one, `(load-network "models/resnet50.nippy" :fc1000 layers-to-add)`

Let's take a closer look at it.

```clojure
(def layers-to-add
  [(layers/linear 2 :id :fc2)
   (layers/softmax :id :labels)])

(defn load-network
  [network-file chop-layer top-layers]
  (let [network (util/read-nippy-file network-file)
        ;; remove last layer(s)
        chopped-net (network/dissoc-layers-from-network network chop-layer)
        ;; set layers to non-trainable
        nodes (get-in chopped-net [:compute-graph :nodes]) ;;=> {:linear-1 {<params>}
        new-node-params (mapv (fn [params] (assoc params :non-trainable? true)) (vals nodes))
        frozen-nodes (zipmap (keys nodes) new-node-params)
        frozen-net (assoc-in chopped-net [:compute-graph :nodes] frozen-nodes)
        ;; add top layers
        modified-net (network/assoc-layers-to-network frozen-net (flatten top-layers))]
    modified-net))
```

What it is doing is taking in that resnet50 trained models and chopping off the last layer and setting all the layers to non-trainable so that the training won't spend any time retraining those layers and leave them frozen. Then it tacks on the layers that we want to add which are the linear layer of size 2 for our 2 classes (cat and dog of classifcation) and a softmax activation layer that will return the probalities that the image is a cat or dog with the values summing to 1.

Let's load up the model and take a look

In [28]:
(require '[cortex.util :as util])

(def plain-res50 (util/read-nippy-file  "models/resnet50.nippy"))

#'cats-dogs-cortex-redux.notebook/plain-res50

The cortex network is just data so we can take a look at the layers

In [31]:
(keys (:compute-graph plain-res50))

(:nodes :edges :buffers :streams)

In [36]:
(clojure.pprint/pprint (sort (keys (get-in plain-res50 [:compute-graph :nodes]))))

(:activation_1
 :activation_10
 :activation_10-split
 :activation_11
 :activation_12
 :activation_13
 :activation_13-split
 :activation_14
 :activation_15
 :activation_16
 :activation_16-split
 :activation_17
 :activation_18
 :activation_19
 :activation_19-split
 :activation_2
 :activation_20
 :activation_21
 :activation_22
 :activation_22-split
 :activation_23
 :activation_24
 :activation_25
 :activation_25-split
 :activation_26
 :activation_27
 :activation_28
 :activation_28-split
 :activation_29
 :activation_3
 :activation_30
 :activation_31
 :activation_31-split
 :activation_32
 :activation_33
 :activation_34
 :activation_34-split
 :activation_35
 :activation_36
 :activation_37
 :activation_37-split
 :activation_38
 :activation_39
 :activation_4
 :activation_4-split
 :activation_40
 :activation_40-split
 :activation_41
 :activation_42
 :activation_43
 :activation_43-split
 :activation_44
 :activation_45
 :activation_46
 :activation_46-split
 :activation_47
 :activation_48
 :activat

There is a function to help print out the network

In [38]:
(require '[cortex.nn.network :as network]
          '[cortex.nn.traverse :as traverse])

In [39]:
(network/print-layer-summary plain-res50 (traverse/training-traversal plain-res50))


|                 type |               input |              output |  :bias | :means | :scale | :variances |    :weights |
|----------------------+---------------------+---------------------+--------+--------+--------+------------+-------------|
|       :convolutional |  3x224x224 - 150528 | 64x112x112 - 802816 |   [64] |        |        |            |    [64 147] |
| :batch-normalization | 64x112x112 - 802816 | 64x112x112 - 802816 |   [64] |   [64] |   [64] |       [64] |             |
|                :relu | 64x112x112 - 802816 | 64x112x112 - 802816 |        |        |        |            |             |
|         :max-pooling | 64x112x112 - 802816 |   64x55x55 - 193600 |        |        |        |            |             |
|               :split |   64x55x55 - 193600 |   64x55x55 - 193600 |        |        |        |            |             |
|       :convolutional |   64x55x55 - 193600 |   64x55x55 - 193600 |   [64] |        |        |            |     [64 64] |
| :batch-normal

Let's see what it looks like after we modify the network:

In [40]:
(def cats-dogs-network (cats-dogs/load-network "models/resnet50.nippy" :fc1000 cats-dogs/layers-to-add))

#'cats-dogs-cortex-redux.notebook/cats-dogs-network

In [41]:
(network/print-layer-summary cats-dogs-network (traverse/training-traversal cats-dogs-network))


|                 type |               input |              output |  :bias | :means | :scale | :variances |    :weights |
|----------------------+---------------------+---------------------+--------+--------+--------+------------+-------------|
|       :convolutional |  3x224x224 - 150528 | 64x112x112 - 802816 |   [64] |        |        |            |    [64 147] |
| :batch-normalization | 64x112x112 - 802816 | 64x112x112 - 802816 |   [64] |   [64] |   [64] |       [64] |             |
|                :relu | 64x112x112 - 802816 | 64x112x112 - 802816 |        |        |        |            |             |
|         :max-pooling | 64x112x112 - 802816 |   64x55x55 - 193600 |        |        |        |            |             |
|       :convolutional |   64x55x55 - 193600 |   64x55x55 - 193600 |   [64] |        |        |            |     [64 64] |
| :batch-normalization |   64x55x55 - 193600 |   64x55x55 - 193600 |   [64] |   [64] |   [64] |       [64] |             |
|              

Notice that the last linear and softmax layers have been replaced. Another point is that the weights are frozen except for the last 2 layers

In [52]:
(clojure.pprint/pprint (->> (map (fn [[name value]] {:name name :non-trainable? (:non-trainable? value)})
                                (get-in cats-dogs-network [:compute-graph :nodes]))
                            (sort-by :non-trainable?)))

({:name :labels, :non-trainable? nil}
 {:name :softmax-loss-1, :non-trainable? nil}
 {:name :fc2, :non-trainable? nil}
 {:name :res2a_branch2b, :non-trainable? true}
 {:name :bn4f_branch2b, :non-trainable? true}
 {:name :bn4e_branch2b, :non-trainable? true}
 {:name :activation_2, :non-trainable? true}
 {:name :res4f_branch2a, :non-trainable? true}
 {:name :activation_29, :non-trainable? true}
 {:name :add_16, :non-trainable? true}
 {:name :activation_48, :non-trainable? true}
 {:name :res4e_branch2b, :non-trainable? true}
 {:name :activation_6, :non-trainable? true}
 {:name :add_9, :non-trainable? true}
 {:name :res2a_branch2a, :non-trainable? true}
 {:name :res4d_branch2a, :non-trainable? true}
 {:name :res2c_branch2b, :non-trainable? true}
 {:name :bn5c_branch2a, :non-trainable? true}
 {:name :add_8, :non-trainable? true}
 {:name :activation_31-split, :non-trainable? true}
 {:name :res5c_branch2c, :non-trainable? true}
 {:name :add_7, :non-trainable? true}
 {:name :add_1, :non-traina

Now we're at a point where we can actually do the training. There is a function called `train` that takes a batch size. If you are running on your computer, and you have memory problems you can try decreasing the batch size or running the core code as an uber jar to do the training. The default is set to a batch size of 32. Another option is running on a AWS P2 compute instance.

For me to be able to run on my old mac, I need to run the uberjar.

If you want to do the uber jar:
* `lein uberjar`
* `java -jar target/cats-dogs-cortex-redux.jar`

Using the GPU on my mac it takes approximately 6 minutes to train 1 epoch **Note: 1 epoch of fine tuning is all we need**

```
Loss for epoch 1: (current) 0.05875186542016347 (best) null
Saving network to trained-network.nippy
```

The key point is that it saved the fine tuned network to `trained-network.nippy`



*Note that we are only going to do 1 epoch of fine tuning*

In [1]:
(ns cats-dogs-cortex-redux.notebook
    (:require [cats-dogs-cortex-redux.core :as cats-dogs]
              [clojure.java.shell :as shell]))

Now we can test out things a bit. There is a `label-one` function that grabs a random image and classifies it

_Note a window will popup with a the dog or cat picture in it_

In [2]:
(cats-dogs/label-one)

{:answer "dog", :guess {:prob 0.9978577494621277, :class "dog"}}

### Step 3 - Run the Kaggle test predictions and get it into csv submission format

You will need to do a bit more setup for this. First, you need to get the Kaggle test images for classification. There are 12500
of these in the `test.zip` file from the site. Under the `data` directory, create a new directory called `kaggle-test`. Now unzip
the contents of test.zip inside that folder. The full directory with all the test images should now be:

`data/kaggle-test/test`

This step takes a long time and you might have to tweak the batch size again depending on your memory. There are _12500_ predications to be made. The main logic for this is in function called `(kaggle-results batch-size)`. It will take a long time to run.
It will print the results as it goes along to the `kaggle-results.csv` file. If you want to check progress you can do ` wc -l kaggle-results.csv`

In [1]:
(ns cats-dogs-cortex-redux.notebook
    (:require [cats-dogs-cortex-redux.core :as cats-dogs]
              [clojure.java.shell :as shell]))

In [2]:
(cats-dogs/kaggle-results 100)

.............................................................................................................................

clojure.lang.LazySeq@f331f79f

Done! It took me about 28 minutes locally. 

Now you can take the `kaggle-results.csv` file and upload it to the competition and check your results!

Mine was `0.10357` **Whoo!**